<a href="https://colab.research.google.com/github/liorZucker11/cloud-computing/blob/main/Shablool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install ipywidgets
#!pip install IPython
#!pip install firebase
from IPython.display import display, HTML
import ipywidgets as widgets

In [ ]:
from firebase import firebase
firebase = firebase.FirebaseApplication('https://cloudproject-shablool-default-rtdb.europe-west1.firebasedatabase.app/', None)

In [ ]:
def getKey(key):
  word_data = firebase.get(f'/index/{key}',None)
  tmp = [word_data['links'],word_data['links_counter']]
  transposed = [[row[i] for row in tmp] for i in range(len(tmp[0]))]
  return transposed

In [ ]:
from ctypes import alignment
import ipywidgets as widgets
from IPython.display import display
import requests

# Function to simulate a search operation and display results
def perform_search(query):
    return [f"Result {i+1} for '{query}'" for i in range(5)]

# Function to update the results display
def update_results_display(search_results):
    results_output.layout.visibility = "visible"
    results_output.clear_output()
    with results_output:
        if search_results:
            for result in search_results:
                print(result)
        else:
            print("No results found.")

# Button actions
def admin_screen_action(b):
    print("Admin Screen action triggered")

def statistics_action(b):
    print("Statistics action triggered")

# Create buttons with actions
admin_screen_button = widgets.Button(description="Admin Screen", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
statistics_button = widgets.Button(description="Statistics", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
admin_screen_button.on_click(admin_screen_action)
statistics_button.on_click(statistics_action)

# Adjust layout for input field, button, and results output
input_layout = widgets.Layout(width='98%', justify_content='center')
button_layout = widgets.Layout(width='98%', justify_content='center', margin='10px 0 0 0')
results_layout = widgets.Layout(border='1px solid black', border_radius='5px', width='95%', margin='20px 0', padding='10px', visibility='hidden')

# Create the search text box, button, and results output
search_input = widgets.Text(value='', placeholder='Type here to search', description='', disabled=False, layout=input_layout)
search_button = widgets.Button(description='SEARCH', button_style='success', tooltip='Click to search', icon='search', layout=button_layout)
results_output = widgets.Output(layout=results_layout)
'''
search_button.on_click(on_search_button_clicked)

# Display logo as a widget
with open("/content/logo.jpg", "rb") as file:  # Update the path to your logo
    image = file.read()
logo = widgets.Image(value=image, format='jpg', layout=widgets.Layout(height='300px', width='auto', display='flex', justify_content='center'))
'''
# Organize the layout
top_buttons = widgets.HBox([admin_screen_button, statistics_button], layout=widgets.Layout(justify_content='flex-start', width='100%'))
vbox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='49%')
##vbox = widgets.VBox([top_buttons, logo, search_input, search_button, results_output], layout=vbox_layout)
vbox = widgets.VBox([top_buttons, search_input, search_button, results_output], layout=vbox_layout)
container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', justify_content='center', height='100vh')
container = widgets.Box(children=[vbox], layout=container_layout)



In [ ]:
import ipywidgets as widgets
from IPython.display import display

def create_admin_screen():
    # Input text box
    input_text = widgets.Text(
        placeholder='Enter text here...',
        description='Query:',
        disabled=False
    )

    # Larger text area for displaying results
    results_text = widgets.Textarea(
        placeholder='Results will be shown here...',
        description='Results:',
        disabled=False,
        layout=widgets.Layout(width='60%', height='200px')
    )

    # Buttons for database operations
    delete_button = widgets.Button(description="Delete", button_style='danger')
    edit_button = widgets.Button(description="Edit", button_style='primary',disabled = True )
    add_button = widgets.Button(description="Add", button_style='success')
    get_button = widgets.Button(description="Get", button_style='info')

    # Actions for buttons can be defined here, for example:
    def on_delete_button_clicked(b):
        # Code to handle delete action
        pass

    def on_edit_button_clicked(b):
        # Code to handle edit action
        pass

    def on_add_button_clicked(b):
        # Code to handle add action
        pass

    def on_get_button_clicked(b):
        edit_button.disabled = False
        results_text.value = '\n'.join([' '.join(map(str, sub_list)) for sub_list in getKey('acquir')])

    # Attach actions to buttons
    delete_button.on_click(on_delete_button_clicked)
    edit_button.on_click(on_edit_button_clicked)
    add_button.on_click(on_add_button_clicked)
    get_button.on_click(on_get_button_clicked)

    # Arrange buttons vertically
    buttons_vbox = widgets.VBox([get_button,add_button,edit_button,delete_button])

    # Create a horizontal layout to place the results area beside the buttons
    hbox = widgets.HBox([results_text, buttons_vbox])

    # Display everything
    display(input_text, hbox)

# Function to be called when the "Admin Screen" button is clicked
def admin_screen_action(b):
    create_admin_screen()

# Assuming admin_screen_button is your previously defined button:
admin_screen_button.on_click(admin_screen_action)

# Now when admin_screen_button is clicked, it will call admin_screen_action
# which will in turn call create_admin_screen to display the new "window"

# Display the elements
display(container)


Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Admin Screen action triggered


Text(value='', description='Query:', placeholder='Enter text here...')

Text(value='', description='Query:', placeholder='Enter text here...')

Text(value='', description='Query:', placeholder='Enter text here...')

Text(value='', description='Query:', placeholder='Enter text here...')

Text(value='', description='Query:', placeholder='Enter text here...')